# Feature extraction

In this notebook we will learn how to extract different features from a text and how to combine them. It's pretty simple, but if you have this part well organized, it will be really useful in the near future. So, let's get started!

In [1]:
import nltk
from sklearn.pipeline import FeatureUnion
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn import preprocessing
from scipy.sparse import coo_matrix, hstack
from copy import deepcopy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import glob
import json
import argparse
import time
import codecs
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

In [34]:
import csv
import re
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, average='macro')

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV




Using TensorFlow backend.


In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import os

In [7]:
def open_file(path):
    with open(path, 'r+', encoding='utf8') as f:
        return '\n'.join([line.strip() for line in f])
    
def process_dir_files(path):
    dir_files = []
    for file in os.listdir(path):
        current = os.path.join(path, file)
        if os.path.isfile(current):
            dir_files.append(open_file(current))
    return dir_files
                     

#train_sents= process_dir_files('pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02/problem00001/candidate00001')



In [8]:
from nltk.tokenize import word_tokenize

def get_pos(text):
    pos_tags= nltk.pos_tag(word_tokenize(text))
    # print(len(pos_tags))
    pos_tags = [word_tag[1] for word_tag in pos_tags]
    pos_text = ' '.join(pos_tags)
    return pos_tags

#get_pos(train_sents[0])

In [9]:
def get_pos_ngrams(sents):
    pos_tags= [nltk.pos_tag(word_tokenize(sents[ind])) for ind, item in enumerate(sents) if item != '']
    pos_sents = []
    for sent in pos_tags:
        #print(sent)
        pos = ' '.join([pos_tag[1] for pos_tag in sent])
        #print(pos, '\n')
        pos_sents.append(pos)
    vectorizer = CountVectorizer(ngram_range = (1,1))
    vectorizer.fit(pos_sents)
    return vectorizer


#pos_vectorizer = get_pos_ngrams(train_sents)
#pos_ngram = pos_vectorizer.transform(train_sents)
#pos_ngram

In [10]:
# -*- coding: utf-8 -*-

"""
 A baseline authorship attribution method 
 based on a character n-gram representation
 and a linear SVM classifier.
 It has a reject option to leave documents unattributed
 (when the probabilities of the two most likely training classes are too close)
 
 Questions/comments: stamatatos@aegean.gr

 It can be applied to datasets of PAN-19 cross-domain authorship attribution task
 See details here: http://pan.webis.de/clef19/pan19-web/author-identification.html
 Dependencies:
 - Python 2.7 or 3.6 (we recommend the Anaconda Python distribution)
 - scikit-learn

 Usage from command line: 
    > python pan19-cdaa-baseline.py -i EVALUATION-DIRECTORY -o OUTPUT-DIRECTORY [-n N-GRAM-ORDER] [-ft FREQUENCY-THRESHOLD] [-pt PROBABILITY-THRESHOLD]
 EVALUATION-DIRECTORY (str) is the main folder of a PAN-19 collection of attribution problems
 OUTPUT-DIRECTORY (str) is an existing folder where the predictions are saved in the PAN-19 format
 Optional parameters of the model:
   N-GRAM-ORDER (int) is the length of character n-grams (default=3)
   FREQUENCY-THRESHOLD (int) is the cutoff threshold used to filter out rare n-grams (default=5)
   PROBABILITY-THRESHOLD (float) is the threshold for the reject option assigning test documents to the <UNK> class (default=0.1)
                                 Let P1 and P2 be the two maximum probabilities of training classes for a test document. If P1-P2<pt then the test document is assigned to the <UNK> class.
   
 Example:

     >  python pan19-cdaa-baseline-svm.py -i ".\pan19-cross-domain-authorship-attribution-training-dataset-2019-01-23\" -o ".\a
nswers-trigram\" -n 3
"""

from __future__ import print_function
import os
import glob
import json
import argparse
import time
import codecs
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.calibration import CalibratedClassifierCV

def represent_text(text,n,pos=False):
    # Extracts all character n-grams from  a 'text'
    # if pos is True, extracts POS n-grams
    if n>0:
        if pos is True:
            text = get_pos(text)
            tokens = [' '.join(text[i:i+n]) for i in range(len(text)-n+1)]
            #print(tokens)
        else:
            tokens = [text[i:i+n] for i in range(len(text)-n+1)]
    frequency = defaultdict(int)
    for token in tokens:
        frequency[token] += 1
    return frequency

#represent_text(train_sents[0], 2)
#represent_text(train_sents[0], 2, pos=True)
                           

In [11]:
def read_files(path,label):
    # Reads all text files located in the 'path' and assigns them to 'label' class
    files = glob.glob(path+os.sep+label+os.sep+'*.txt')
    texts=[]
    for i,v in enumerate(files):
        f=codecs.open(v,'r',encoding='utf-8')
        texts.append((f.read(),label))
        f.close()
    return texts

def extract_vocabulary(texts,n,ft,pos=False):
    # Extracts all characer 'n'-grams occurring at least 'ft' times in a set of 'texts'
    occurrences=defaultdict(int) 
    for (text,label) in texts:
        text_occurrences = {}
        if isinstance(n, int):
            for x in range(1,n+1):
                text_occurrences.update(represent_text(text,x,pos=pos))
        else:
            pass
        for ngram in text_occurrences:
            if ngram in occurrences:
                occurrences[ngram]+=text_occurrences[ngram]
            else:
                occurrences[ngram]=text_occurrences[ngram]
    vocabulary=[]
    for i in occurrences.keys():
        if occurrences[i]>=ft:
            vocabulary.append(i)
    return vocabulary

'''
vocab = extract_vocabulary([(x,i) for i, x in enumerate(train_sents)], 2, 5, pos=True)
print(len(vocab))
print(vocab)
vectorizer = CountVectorizer(vocabulary=[x.lower() for  x in vocab])
print([' '.join(get_pos(text)) for text in train_sents])

train_data = vectorizer.fit_transform([' '.join(get_pos(text)) for text in train_sents])
print(vectorizer.get_feature_names())
train_data = train_data.astype(float)
print(train_data.shape)
print(train_data.toarray())
'''

"\nvocab = extract_vocabulary([(x,i) for i, x in enumerate(train_sents)], 2, 5, pos=True)\nprint(len(vocab))\nprint(vocab)\nvectorizer = CountVectorizer(vocabulary=[x.lower() for  x in vocab])\nprint([' '.join(get_pos(text)) for text in train_sents])\n\ntrain_data = vectorizer.fit_transform([' '.join(get_pos(text)) for text in train_sents])\nprint(vectorizer.get_feature_names())\ntrain_data = train_data.astype(float)\nprint(train_data.shape)\nprint(train_data.toarray())\n"

In [12]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [13]:
STOPWORDS = list(stopwords.words('english'))
len(STOPWORDS)

179

In [14]:
lex_div = np.zeros((5,1))
lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
lex_div

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [15]:
from sklearn.base import TransformerMixin #gives fit_transform method for free

class Feature_Extractor(TransformerMixin):
    '''
    Performs feature extraction on input docs.
    '''
    def __init__(self, char=True, pos=True, word=True, lexdiv=True, fnword=True):
        self.char = char # uses character ngrams
        self.pos = pos # uses pos ngrams
        self.word = word # uses word ngrams
        self.fnword = fnword # uses function word counts
        self.lexdiv = lexdiv # uses lexical diversity score
        
    def set_params(self, char, pos, word, lexdiv, fnword):
        self.char = char # uses character ngrams
        self.pos = pos # uses pos ngrams
        self.word = word # uses word ngrams
        self.fnword = fnword # uses function word counts
        self.lexdiv = lexdiv # uses lexical diversity score

        
    def fit_transform(self, docs, y=None):
        ## Char-level n-grams ##
        feature_list = []
        self.texts = [text for i,(text,label) in enumerate(docs)]
        print('Extracting features from train data')
        if self.lexdiv is True or not any((self.char, self.pos, self.word, self.fnword, self.lexdiv)) :
            ## Lexical Diversity
            lex_div = self.lexical_diversity(docs)
            if not any((self.char, self.pos, self.word, self.fnword)):
                lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
            feature_list.append(lex_div)
            
        if self.char is True:
            char_vocab = extract_vocabulary(docs, 4, 3)
            self.char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(4,4),
                                              lowercase=False, vocabulary=char_vocab
                                                  )
            char_data, self.char_vectorizer = self._fit_transform(self.char_vectorizer)
            print('char vocabulary size:', len(char_vocab))
            feature_list.append(char_data)
            # print(char_data.toarray())
        
        ## POS n-grams ##
        if self.pos is True:
            pos_vocab = [x.lower() for x in extract_vocabulary(docs,2,3,pos=True)]
            # print(pos_vocab)
            self.pos_vectorizer = CountVectorizer(ngram_range=(1,3), vocabulary=pos_vocab
                                                 )
            print('\t', 'pos vocabulary size:', len(pos_vocab))
            pos_data, self.pos_vectorizer = self._fit_transform(self.pos_vectorizer, pos_replace=True)
            feature_list.append(pos_data)

        ## Word n-grams ##
        if self.word is True:
            self.word_vectorizer = CountVectorizer(ngram_range=(1,1))
            word_data, self.word_vectorizer = self._fit_transform(self.word_vectorizer)
            feature_list.append(word_data)
        
        ## Function Word counts ##
        if self.fnword is True:
            self.fnword_vectorizer = CountVectorizer(vocabulary=STOPWORDS)
            fnword_data, self.fnword_vectorizer = self._fit_transform(self.fnword_vectorizer)
            feature_list.append(fnword_data)

        print('lexdiv: %s char: %s pos: %s word: %s fnword: %s'%(self.lexdiv, self.char, self.pos, 
                                                                          self.word, self.fnword))
        feature_data = self.combine_features(tuple(feature_list)) 
        print('size of features:', feature_data.shape)   
        return feature_data
    
    def combine_features(self, feat_tuple):
        feature_data = hstack(feat_tuple)
        return feature_data
    
    def replace_words_POS(self, texts):
        return [' '.join(get_pos(text)) for text in texts]
    
    def lexical_diversity(self, docs):
        lex_div = np.array([len(set(text)) / len(text) for (text,label) in docs]).reshape(len(docs), 1)
        # print('lexical diversity:', lex_div.shape)
        return lex_div
    
    def _fit_transform(self, vectorizer, pos_replace=False):
        if pos_replace is True:
            texts = self.replace_words_POS(self.texts) # replace words in text with POS
        else:
            texts = self.texts
        vec_data = vectorizer.fit_transform(texts)
        vec_data = vec_data.astype(float)
        return vec_data, vectorizer
    
    def _transform(self, vectorizer, pos_replace=False):
        if pos_replace is True:
            texts = self.replace_words_POS(self.texts) # replace words in text with POS
        else:
            texts = self.texts
        vec_data = vectorizer.transform(texts)
        vec_data = vec_data.astype(float)
        return vec_data
    
    def transform(self, docs):
        feature_list = []
        self.texts = [text for i,(text,label) in enumerate(docs)]
        if self.lexdiv is True or not any((self.char, self.pos, self.word, self.fnword, self.lexdiv)):
            lex_div = self.lexical_diversity(docs)
            if not any((self.char, self.pos, self.word, self.fnword)):
                lex_div = np.hstack((lex_div, np.ones(lex_div.shape)))
            feature_list.append(lex_div)
        if self.char is True:
            feature_list.append(self._transform(self.char_vectorizer))
        if self.word is True:
            feature_list.append(self._transform(self.word_vectorizer))
        if self.pos is True:
            feature_list.append(self._transform(self.pos_vectorizer, pos_replace=True))
        if self.fnword is True:
            feature_list.append(self._transform(self.fnword_vectorizer))
        print('Extracting features from test data')
        print('lexdiv: %s char: %s pos: %s word: %s fnword: %s'%(self.lexdiv, self.char, self.pos, 
                                                                          self.word, self.fnword))
        feature_data = self.combine_features(tuple(feature_list)) 
        print('size of features:', feature_data.shape)   
        return feature_data
        
def write_results(path, problem, unk_folder, predictions):
    # Saving output data
    out_data=[]
    unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
    pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
    for i,v in enumerate(predictions):
        out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
    with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
        json.dump(out_data, f, indent=4)
    print('\t', 'answers saved to file','answers-'+problem+'.json')
    

def get_baseline_model_fn(num_in, num_out):
    # for Keras: initialize baseline model fn with num features and num predicted categories
    def baseline_model():
        # create model
        model = Sequential()
        model.add(Dense(8, input_dim=num_in, activation='relu'))
        model.add(Dense(num_out, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    return baseline_model

In [59]:

def pipeline_svm(path, outpath):
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    scores_all = []
    param_grids = [{
        'features__char': [True],
        'features__pos':  [False],
        'features__word':  [True],
        'features__lexdiv':  [True],
        'features__fnword':  [True],
        'sel__percentile': [60],
        },
        {
        'features__char':  [True],
        'features__pos':  [True],
        'features__word':  [False],
        'features__lexdiv':  [True],
        'features__fnword':  [True],
        'sel__percentile': [40],
        }
    ]
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        svm =  CalibratedClassifierCV(SVC(random_state=442))
        feat_extractor = Feature_Extractor()
        sel = SelectPercentile(f_classif)
        scaler = preprocessing.MaxAbsScaler()
        pipe = Pipeline(steps=[('features', feat_extractor), ('sel', sel), 
                               ('scale', scaler), ('svm', svm)])
        param_grid = param_grids[index]
        search = GridSearchCV(pipe, param_grid, iid=False, cv=5, n_jobs=3, verbose=10,
                             scoring=f1_scorer)
        search.fit(docs, train_labels)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        scores_all.append(search.cv_results_)
    return scores_all


%timeit

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
svm_results = pipeline_svm(base_dir,out_dir)
svm_results

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 out of   5 | elapsed:    2.6s remaining:    4.0s
[Parallel(n_jobs=3)]: Done   3 out of   5 | elapsed:    3.1s remaining:    2.1s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    5.1s finished


Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: True
size of features: (140, 38084)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Best parameter (CV score=0.595):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 60, 'svm__C': 1}
problem00002
	 language:  en
	 5 candidate authors
	 35 known texts
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 out of   5 | elapsed:    5.2s remaining:    7.9s
[Parallel(n_jobs=3)]: Done   3 out of   5 | elapsed:    6.0s remaining:    4.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:   10.9s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:   10.9s finished


Extracting features from train data
char vocabulary size: 14415
	 pos vocabulary size: 619
lexdiv: True char: True pos: True word: False fnword: True
size of features: (35, 15214)
Best parameter (CV score=0.905):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': True, 'features__word': False, 'sel__percentile': 40, 'svm__C': 1}


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/dep

[{'mean_fit_time': array([1.48996987]),
  'std_fit_time': array([0.19343633]),
  'mean_score_time': array([0.14955211]),
  'std_score_time': array([0.04436804]),
  'param_features__char': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__fnword': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__lexdiv': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__pos': masked_array(data=[False],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_features__word': masked_array(data=[True],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_sel__percentile': masked_array(data=[60],
               mask=[False],
         fill_value='?',
              dtype=object),
  'param_svm__C': masked_

In [49]:
pd.DataFrame(svm_results[0]).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,2.030841,0.18204,0.24922,0.095935,True,True,True,False,True,60,...,0.692048,0.077611,1,0.979798,0.937374,0.949291,0.956773,0.947169,0.954081,0.014272


In [52]:
pd.DataFrame(svm_results[1]).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,4.192103,0.385163,0.346333,0.122864,True,True,True,True,False,40,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,3.881361,0.276173,0.332125,0.105892,True,True,True,True,False,50,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,3.897903,0.352293,0.339335,0.111427,True,True,True,True,False,60,...,0.960000,0.080000,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,3.926167,0.358136,0.336105,0.111188,True,True,True,True,False,70,...,0.960000,0.080000,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,4.161879,0.451723,0.361942,0.107644,True,True,True,True,False,80,...,0.960000,0.080000,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,4.165527,0.214032,0.353655,0.126025,True,True,True,True,False,90,...,0.960000,0.080000,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [53]:
#pd.DataFrame(svm_results[0]).to_csv('problem1-allfeatures_svm.csv')
#pd.DataFrame(svm_results[1]).to_csv('problem2-allfeatures_svm.csv')
#pd.DataFrame(svm_results[0]).to_csv('problem1-featsel_svm.csv')
#pd.DataFrame(svm_results[1]).to_csv('problem2-featsel_svm.csv')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [71]:
pd.DataFrame(svm_results[1]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,3.816941,0.254186,0.318367,0.107445,True,True,True,True,False,65,...,0.98,0.04,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [112]:

def pipeline_perceptron(path, outpath):
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    scores_all = []
    param_grids = [{
        'features__char': [True],
        'features__pos': [False],
        'features__word': [True],
        'features__lexdiv': [True],
        'features__fnword': [False],
        'sel__percentile':[80], # 80
        },
        {
        'features__char':[True],
        'features__pos': [False],
        'features__word': [True],
        'features__lexdiv': [True],
        'features__fnword': [False],
        'sel__percentile': [70],  # 70 : 1.0 score
        }
    ]
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        clf = CalibratedClassifierCV(MLPClassifier(random_state=442))
        feat_extractor = Feature_Extractor()
        sel = SelectPercentile(f_classif)
        scaler = preprocessing.MaxAbsScaler()
        pipe = Pipeline(steps=[('features', feat_extractor), ('sel', sel), 
                               ('scale', scaler), ('clf', clf)])
        param_grid = param_grids[index]
        search = GridSearchCV(pipe, param_grid, iid=False, cv=5, n_jobs=3, verbose=10,
                             scoring=f1_scorer)
        search.fit(docs, train_labels)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        scores_all.append(search.cv_results_)
    return scores_all


%timeit

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
perceptron_results = pipeline_perceptron(base_dir,out_dir)
perceptron_results

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   10.8s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   36.7s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   51.4s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:  2.3min finished


Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: False
size of features: (140, 37905)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameter (CV score=0.674):
{'features__char': True, 'features__fnword': False, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 80}
problem00002
	 language:  en
	 5 candidate authors
	 35 known texts
Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

Extracting features from train data
char vocabulary size: 14415
lexdiv: True char: True pos: False word: True fnword: False
size of features: (35, 18699)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameter (CV score=1.000):
{'features__char': True, 'features__fnword': False, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 70}


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

[{'mean_fit_time': array([11.28339696, 12.28421683, 12.35871873, 12.73991919, 13.80846243,
         13.21004753]),
  'std_fit_time': array([1.86879221, 0.80877482, 0.84035798, 0.47859643, 0.88130397,
         1.90304476]),
  'mean_score_time': array([0.17243304, 0.15335865, 0.13326769, 0.16669688, 0.17421265,
         0.1811337 ]),
  'std_score_time': array([0.07757541, 0.04769299, 0.04389529, 0.06104827, 0.06038226,
         0.08122814]),
  'param_features__char': masked_array(data=[True, True, True, True, True, True],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_features__fnword': masked_array(data=[False, False, False, False, False, False],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_features__lexdiv': masked_array(data=[True, True, True, True, True, True],
               mask=[False, False, False, False, False, False],


In [113]:
pd.DataFrame(perceptron_results[0]).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
4,13.808462,0.881304,0.174213,0.060382,True,False,True,False,True,80,...,0.674048,0.076518,1,1.0,0.989899,1.0,1.0,1.000000,0.997980,0.004040
3,12.739919,0.478596,0.166697,0.061048,True,False,True,False,True,70,...,0.638714,0.068735,2,1.0,1.000000,1.0,1.0,1.000000,1.000000,0.000000
5,13.210048,1.903045,0.181134,0.081228,True,False,True,False,True,90,...,0.601636,0.054770,3,1.0,1.000000,1.0,1.0,0.945604,0.989121,0.021758
1,12.284217,0.808775,0.153359,0.047693,True,False,True,False,True,50,...,0.547190,0.048370,4,1.0,1.000000,1.0,1.0,1.000000,1.000000,0.000000
2,12.358719,0.840358,0.133268,0.043895,True,False,True,False,True,60,...,0.531516,0.083968,5,1.0,1.000000,1.0,1.0,1.000000,1.000000,0.000000
0,11.283397,1.868792,0.172433,0.077575,True,False,True,False,True,40,...,0.467587,0.060115,6,1.0,1.000000,1.0,1.0,1.000000,1.000000,0.000000


In [114]:
pd.DataFrame(perceptron_results[1]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,4.457396,0.785217,0.067822,0.021043,True,False,True,False,True,70,...,1.000000,0.000000,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,3.651717,0.387354,0.075955,0.039438,True,False,True,False,True,50,...,0.946667,0.106667,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,5.041605,0.549437,0.092314,0.054048,True,False,True,False,True,80,...,0.946667,0.106667,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,4.527471,0.384782,0.067332,0.023328,True,False,True,False,True,60,...,0.925333,0.104512,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,5.025832,0.735824,0.064129,0.024347,True,False,True,False,True,90,...,0.906667,0.116237,5,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,3.214342,0.306537,0.075612,0.023011,True,False,True,False,True,40,...,0.833333,0.217051,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [ ]:
perceptron_results[1]

In [75]:

perceptron_results[0] = pd.concat([df, pd.DataFrame(perceptron_results[0])]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [77]:
df = pd.read_csv('problem2-allfeatures_perceptron.csv')
df

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0,9.378188,0.566679,0.540325,0.303743,True,True,True,True,True,...,0.206545,0.128651,14,1.000000,0.959596,0.664348,1.000000,1.000000,0.924789,0.131157
1,1,10.028698,2.525919,0.694575,0.418177,True,True,True,True,False,...,0.886667,0.139204,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,2,5.004416,0.826440,0.055383,0.017068,True,True,True,False,True,...,0.878667,0.193788,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,3,4.348181,0.736264,0.068432,0.019715,True,True,True,False,False,...,0.958667,0.082667,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,4,9.693593,0.946180,0.560989,0.228317,True,True,False,True,True,...,0.306667,0.199332,13,0.916667,0.959596,1.000000,1.000000,0.966434,0.968539,0.030837
5,5,8.794175,1.667437,0.532886,0.125308,True,True,False,True,False,...,0.840000,0.130639,6,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,6,5.364172,0.950844,0.052829,0.030473,True,True,False,False,True,...,0.872000,0.119733,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
7,7,5.751495,0.353122,0.058469,0.024981,True,True,False,False,False,...,0.957333,0.085333,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,8,5.510895,0.765170,0.450963,0.136863,False,True,True,True,True,...,0.131152,0.094941,15,0.055172,0.066667,0.068571,0.155000,0.252381,0.119558,0.075420
9,9,5.349497,1.169149,0.378952,0.100753,False,True,True,True,False,...,0.776000,0.159867,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [78]:
perceptron_results[1] = pd.concat([df, pd.DataFrame(perceptron_results[1])]).sort_values('mean_test_score', ascending=False)
perceptron_results[1]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,Unnamed: 0,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,NaN,5.369243,0.049326,1.000000,1.000000,True,False,True,False,True,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.202269,0.018782,0.000000,0.000000
3,3.0,4.348181,0.068432,0.958667,1.000000,True,True,True,False,False,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.736264,0.019715,0.082667,0.000000
7,7.0,5.751495,0.058469,0.957333,1.000000,True,True,False,False,False,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.353122,0.024981,0.085333,0.000000
1,1.0,10.028698,0.694575,0.886667,1.000000,True,True,True,True,False,...,1.000000,1.000000,0.733333,1.000000,1.000000,1.000000,2.525919,0.418177,0.139204,0.000000
2,2.0,5.004416,0.055383,0.878667,1.000000,True,True,True,False,True,...,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.826440,0.017068,0.193788,0.000000
0,NaN,6.475509,0.075786,0.878667,1.000000,True,True,True,False,True,...,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.401081,0.021937,0.193788,0.000000
6,6.0,5.364172,0.052829,0.872000,1.000000,True,True,False,False,True,...,0.733333,1.000000,0.733333,1.000000,1.000000,1.000000,0.950844,0.030473,0.119733,0.000000
5,5.0,8.794175,0.532886,0.840000,1.000000,True,True,False,True,False,...,0.733333,1.000000,1.000000,1.000000,0.733333,1.000000,1.667437,0.125308,0.130639,0.000000
14,14.0,1.461205,0.038941,0.830667,1.000000,False,True,False,False,True,...,1.000000,1.000000,0.733333,1.000000,0.733333,1.000000,0.123569,0.013101,0.102900,0.000000
13,13.0,5.840288,0.504886,0.796000,1.000000,False,True,False,True,False,...,0.733333,1.000000,1.000000,1.000000,0.733333,1.000000,1.865169,0.090401,0.105122,0.000000


In [68]:
pd.DataFrame(perceptron_results[1]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,5.369243,1.202269,0.049326,0.018782,True,False,True,False,True,"{'features__char': True, 'features__fnword': F...",...,1.000000,0.000000,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,6.475509,0.401081,0.075786,0.021937,True,True,True,False,True,"{'features__char': True, 'features__fnword': T...",...,0.878667,0.193788,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [115]:
#pd.DataFrame(perceptron_results[0]).to_csv('problem1-allfeatures_perceptron.csv')
#pd.DataFrame(perceptron_results[1]).to_csv('problem2-allfeatures_perceptron.csv')
pd.DataFrame(perceptron_results[0]).to_csv('problem1-featsel_perceptron.csv')
pd.DataFrame(perceptron_results[1]).to_csv('problem2-featsel_perceptron.csv')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [127]:

def pipeline_logreg(path, outpath):
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    scores_all = []
    param_grids = [{
        'features__char': [True],
        'features__pos': [False],
        'features__word': [True],
        'features__lexdiv': [True],
        'features__fnword': [True],
        'sel__percentile': [10, 20, 30, 40, 50, 60, 70, 80, 90],
        },
        {
        'features__char': [True],
        'features__pos': [False],
        'features__word': [True],
        'features__lexdiv': [True],
        'features__fnword': [True],
        'sel__percentile': [10, 20, 30, 40, 50, 60, 70, 80, 90],
        }
    ]
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        clf = CalibratedClassifierCV(LogisticRegression(random_state=442, multi_class='multinomial',
                                                       solver= 'lbfgs'))
        feat_extractor = Feature_Extractor()
        sel = SelectPercentile(f_classif)
        scaler = preprocessing.MaxAbsScaler()
        pipe = Pipeline(steps=[('features', feat_extractor), ('sel', sel), 
                               ('scale', scaler), ('clf', clf)])
        param_grid = param_grids[index]
        search = GridSearchCV(pipe, param_grid, iid=False, cv=5, n_jobs=3, verbose=10,
                             scoring=f1_scorer)
        search.fit(docs, train_labels)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        scores_all.append(search.cv_results_)
    return scores_all


%timeit

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
logreg_results = pipeline_logreg(base_dir,out_dir)
logreg_results

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    5.4s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   17.8s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   32.4s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  1.8min
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  2.7min
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:  3.9min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:  3.9min finished


Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: True
size of features: (140, 38084)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameter (CV score=0.812):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 70}
problem00002
	 language:  en
	 5 candidate authors
	 35 known texts
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    2.4s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    4.0s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:    8.1s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   13.4s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   20.0s
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:   27.9s remaining:    0.0s
[Parallel(n_jobs=3)]: Done  45 out of  45 | elapsed:   27.9s finished


Extracting features from train data
char vocabulary size: 14415
lexdiv: True char: True pos: False word: True fnword: True
size of features: (35, 18878)
Best parameter (CV score=0.979):
{'features__char': True, 'features__fnword': True, 'features__lexdiv': True, 'features__pos': False, 'features__word': True, 'sel__percentile': 10}


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning

[{'mean_fit_time': array([ 4.31878672,  7.77193532, 10.56760583, 12.35927987, 15.11613183,
         17.29636035, 18.01697712, 21.52822928, 20.45061102]),
  'std_fit_time': array([0.66737792, 1.45151519, 0.65976632, 2.6359256 , 2.06390381,
         0.89379519, 3.8028027 , 3.60272143, 2.64542597]),
  'mean_score_time': array([0.23439541, 0.25781713, 0.18837056, 0.23459167, 0.23265605,
         0.20923896, 0.22211709, 0.29447336, 0.19499512]),
  'std_score_time': array([0.10825074, 0.12283958, 0.04577703, 0.09324129, 0.13686446,
         0.0936905 , 0.04145277, 0.1602363 , 0.08877826]),
  'param_features__char': masked_array(data=[True, True, True, True, True, True, True, True, True],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_features__fnword': masked_array(data=[True, True, True, True, True, True, True, True, True],
               mask=[False, False, False, False

In [128]:
pd.DataFrame(logreg_results[0]).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
6,18.016977,3.802803,0.222117,0.041453,True,True,True,False,True,70,...,0.812000,0.079822,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
7,21.528229,3.602721,0.294473,0.160236,True,True,True,False,True,80,...,0.800333,0.084066,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
8,20.450611,2.645426,0.194995,0.088778,True,True,True,False,True,90,...,0.800333,0.084066,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,15.116132,2.063904,0.232656,0.136864,True,True,True,False,True,50,...,0.774048,0.063727,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,17.296360,0.893795,0.209239,0.093691,True,True,True,False,True,60,...,0.761381,0.051239,5,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,12.359280,2.635926,0.234592,0.093241,True,True,True,False,True,40,...,0.747381,0.086181,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,10.567606,0.659766,0.188371,0.045777,True,True,True,False,True,30,...,0.744381,0.088859,7,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,7.771935,1.451515,0.257817,0.122840,True,True,True,False,True,20,...,0.680714,0.047889,8,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,4.318787,0.667378,0.234395,0.108251,True,True,True,False,True,10,...,0.643667,0.079207,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [129]:
pd.DataFrame(logreg_results[1]).sort_values('mean_test_score', ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,param_sel__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.470389,0.022207,0.031639,0.008402,True,True,True,False,True,10,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.888377,0.301100,0.066707,0.027608,True,True,True,False,True,20,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,1.110735,0.197070,0.073168,0.025523,True,True,True,False,True,30,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,1.471456,0.417891,0.067280,0.027842,True,True,True,False,True,40,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,1.769562,0.187808,0.077564,0.031828,True,True,True,False,True,50,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,1.882119,0.247957,0.086687,0.041918,True,True,True,False,True,60,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
6,2.078898,0.578739,0.072737,0.017664,True,True,True,False,True,70,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
7,2.159557,0.173692,0.092796,0.049031,True,True,True,False,True,80,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
8,2.055774,0.300539,0.076021,0.037594,True,True,True,False,True,90,...,0.978667,0.042667,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [103]:
df = pd.read_csv('problem1-allfeatures_logreg.csv')
logreg_results[0] = pd.concat([df, pd.DataFrame(logreg_results[0])]).sort_values('mean_test_score', ascending=False)
logreg_results[0]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,Unnamed: 0,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,NaN,23.895635,0.186760,0.800333,1.000000,True,False,False,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,3.154712,0.102936,0.084066,0.000000
2,2.0,21.936970,0.185608,0.800333,1.000000,True,True,True,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,6.108673,0.041154,0.084066,0.000000
0,NaN,28.350842,0.199160,0.800333,1.000000,True,True,False,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,4.211949,0.047478,0.084066,0.000000
6,6.0,21.237986,0.198145,0.800333,1.000000,True,True,False,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,5.363950,0.014042,0.084066,0.000000
3,3.0,21.533731,0.134763,0.796048,1.000000,True,True,True,False,False,...,0.866667,1.000000,0.866667,1.000000,0.816667,1.000000,2.686781,0.047722,0.068584,0.000000
7,7.0,18.701316,0.204699,0.782714,1.000000,True,True,False,False,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,2.452363,0.116511,0.059428,0.000000
5,5.0,39.844418,2.133967,0.769381,1.000000,True,True,False,True,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,6.918241,1.195326,0.075277,0.000000
1,1.0,40.643819,2.284577,0.769381,1.000000,True,True,True,True,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,6.830647,1.296193,0.075277,0.000000
10,10.0,3.572286,0.074990,0.587000,0.970839,False,True,True,False,True,...,0.608333,0.926923,0.591667,1.000000,0.483333,1.000000,0.974147,0.022353,0.067044,0.035715
14,14.0,3.538467,0.089891,0.587000,0.970839,False,True,False,False,True,...,0.608333,0.926923,0.591667,1.000000,0.483333,1.000000,0.544701,0.038699,0.067044,0.035715


In [104]:
df = pd.read_csv('problem1-allfeatures_logreg.csv')
logreg_results[1] = pd.concat([df, pd.DataFrame(logreg_results[1])]).sort_values('mean_test_score', ascending=False)
logreg_results[1]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

,Unnamed: 0,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_features__char,param_features__fnword,param_features__lexdiv,param_features__pos,param_features__word,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,NaN,2.023231,0.048403,0.978667,1.000000,True,False,False,False,False,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.431045,0.027479,0.042667,0.000000
0,NaN,2.117552,0.073544,0.978667,1.000000,True,True,False,False,False,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.240263,0.032245,0.042667,0.000000
2,2.0,21.936970,0.185608,0.800333,1.000000,True,True,True,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,6.108673,0.041154,0.084066,0.000000
6,6.0,21.237986,0.198145,0.800333,1.000000,True,True,False,False,True,...,0.933333,1.000000,0.866667,1.000000,0.733333,1.000000,5.363950,0.014042,0.084066,0.000000
3,3.0,21.533731,0.134763,0.796048,1.000000,True,True,True,False,False,...,0.866667,1.000000,0.866667,1.000000,0.816667,1.000000,2.686781,0.047722,0.068584,0.000000
7,7.0,18.701316,0.204699,0.782714,1.000000,True,True,False,False,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,2.452363,0.116511,0.059428,0.000000
5,5.0,39.844418,2.133967,0.769381,1.000000,True,True,False,True,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,6.918241,1.195326,0.075277,0.000000
1,1.0,40.643819,2.284577,0.769381,1.000000,True,True,True,True,False,...,0.800000,1.000000,0.866667,1.000000,0.816667,1.000000,6.830647,1.296193,0.075277,0.000000
10,10.0,3.572286,0.074990,0.587000,0.970839,False,True,True,False,True,...,0.608333,0.926923,0.591667,1.000000,0.483333,1.000000,0.974147,0.022353,0.067044,0.035715
14,14.0,3.538467,0.089891,0.587000,0.970839,False,True,False,False,True,...,0.608333,0.926923,0.591667,1.000000,0.483333,1.000000,0.544701,0.038699,0.067044,0.035715


In [130]:
#pd.DataFrame(logreg_results[0]).to_csv('problem1-allfeatures_logreg.csv')
#pd.DataFrame(logreg_results[1]).to_csv('problem2-allfeatures_logreg.csv')
pd.DataFrame(logreg_results[0]).to_csv('problem1-featsel_logreg.csv')
pd.DataFrame(logreg_results[1]).to_csv('problem2-featsel_logreg.csv')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [157]:
df_char_1 = pd.read_csv('problem1-charngrams.csv')
df_char_1range = pd.read_csv('problem1-charngramsrange.csv')
df_char_2 = pd.read_csv('problem2-charngrams.csv')
df_char_2range = pd.read_csv('problem2-charngramsrange.csv')
np.mean([pd.concat([df_char_1,df_char_1range])['mean_test_score'], pd.concat([df_char_2,df_char_2range])['mean_test_score']], axis=0)


array([0.565 , 0.63  , 0.7925, 0.8225, 0.785 , 0.7975, 0.835 , 0.825 ])

In [30]:
def baseline_keras(path, outpath, n=3, ft=5, pt=0.1, feature_selection=False, 
             open_set=False, c=1, feat_sel_percent=None, clf=None, calibration=True):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    problem_scores = []
    for index,problem in enumerate(problems):
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        docs=[]
        for candidate in candidates:
            docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = np.array([label for i,(text,label) in enumerate(docs)])
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(docs), 'known texts')
        
        ###### Applying Classifiers #####
        if calibration is True:
            clf=CalibratedClassifierCV(OneVsRestClassifier(SVC(C=c)))
        else:
            clf=OneVsRestClassifier(SVC(C=c))
        skf = StratifiedKFold(n_splits=5,random_state=442)
        scores = []
        print("Do cross validation.")
        for train_index, test_index in skf.split(docs, train_labels):
            X_train_docs = [docs[i] for i in train_index]
            X_test_docs = [docs[i] for i in test_index]
            y_train, y_test = train_labels[train_index], train_labels[test_index]
            ##### Extract X features ####
            feat_extractor = Feature_Extractor(n, ft)
            X_train = feat_extractor.fit_transform(X_train_docs)
            X_test = feat_extractor.transform(X_test_docs)
            print('train shape:', X_train.shape, 'test shape:', X_test.shape)
            if feature_selection is True:
                ####### Feature Selection - Fit #######
                print("training before feature selection:", X_train.shape)
                print("testing before feature selection:", X_test.shape)
                #sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
                #train_data = sel.fit_transform(train_data)
                # We use the default selection function: the 10% most significant features
                sel = SelectPercentile(f_classif, percentile=feat_sel_percent)
                X_train = sel.fit_transform(X_train, y_train)
                X_test = sel.transform(X_test)
                #sel = SelectKBest(chi2, k=100000)
                #train_data = sel.fit_transform(train_data, train_labels)
                print("training after feature selection:", X_train.shape)
                print("testing after feature selection:", X_test.shape)
            max_abs_scaler = preprocessing.MaxAbsScaler()
            X_train = max_abs_scaler.fit_transform(X_train)
            X_test = max_abs_scaler.transform(X_test)
            baseline_model = get_baseline_model_fn(X_train.shape[1], len(candidates))
            clf = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
            ##### Convert y to 1-hot encoding ####
            y_encoder = LabelEncoder()
            y_train = y_encoder.fit_transform(y_train)
            # convert integers to dummy variables (i.e. one hot encoded)
            y_train = np_utils.to_categorical(y_train)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print("y_pred y_test:", y_pred.shape, y_test.shape)
            y_pred = y_encoder.inverse_transform(y_pred)
            print(y_pred)
            print(y_test)
            accuracy = accuracy_score(y_test, y_pred)
            scores.append(accuracy)
            print('accuracy:', accuracy)
            #write_results(path, problem, unk_folder, predictions)
        mean_score = np.mean(scores)
        print(problem, 'MEAN ACCURACY SCORES:', mean_score)
        problem_scores.append(mean_score)
    print('MEAN SCORES ACCROSS PROBLEMS:', np.mean(problem_scores))
    # todo: also add stdev of scores
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': False, 'c':0.1, 'feat_sel_percent': None, 'clf': 'SVC'}
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'SVC', 'calibration': True}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':10, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline_keras(base_dir,out_dir,**params)
%timeit
    

problem00001
	 language:  en
	 20 candidate authors
	 140 known texts
Do cross validation.
(100, 52773)
	 pos vocabulary size: 806 char vocabulary size: 52773
lexical diversity: (100, 1)
[ 21.  25.   6.  37.  39.  52.  16.  48.  59.  49.  99.  85.  47.  55.
  49.  50.  37.  31.  42.  61.  87.  31.  40.  24.  59.  46.  43.  68.
  47.  85.  42. 120.  94. 110. 127. 107.  31.  59. 108.  95.  77.  80.
  75. 118.  98.  32.  54.  46.  64.  46.  54.  47.  86.  52.  68.  78.
  57.  77.  90.  77.  62.  54.  55.  74.  86.  55.  55.  34.  61.  28.
  70.  57.  44.  40.  66.  16.  13.  42.  21.  68.  62.  42. 105.  84.
  38.  92.  93.  91.  87. 107.  45.  88.  57. 103.  65.  67.  38.  49.
  73.  76.]
pos data: (100, 806) char data: (100, 52773) word data: (100, 116893)
lexical diversity: (40, 1)
train shape: (100, 170473) test shape: (40, 170473)
training before feature selection: (100, 170473)
testing before feature selection: (40, 170473)
training after feature selection: (100, 144902)
testing aft

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (40,) (40,)
['candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00012' 'candidate00012' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00013' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013'
 'candidate00013' 'candidate00013' 'candidate00012' 'candidate00013']
['candidate00001' 'candidate00001' 'candidate00002' 'candidate00002'
 'candidate00003' 'candidate00003' 'candidate00004' 'candidate00004'
 'candidate00005' 'candidate00005' 'candidate00006' 'candidate00006'
 'candidate00007' 'candidate00007' 'candidate00008' 'candidate00008'
 'cand

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (40,) (40,)
['candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00014' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005'
 'candidate00005' 'candidate00005' 'candidate00005' 'candidate00005']
['candidate00001' 'candidate00001' 'candidate00002' 'candidate00002'
 'candidate00003' 'candidate00003' 'candidate00004' 'candidate00004'
 'candidate00005' 'candidate00005' 'candidate00006' 'candidate00006'
 'candidate00007' 'candidate00007' 'candidate00008' 'candidate00008'
 'cand

...

y_pred y_test: (5,) (5,)
['candidate00001' 'candidate00005' 'candidate00003' 'candidate00001'
 'candidate00005']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.6
(30, 23884)
	 pos vocabulary size: 592 char vocabulary size: 23884
lexical diversity: (30, 1)
[29. 40. 36. 24. 41. 29. 12. 28. 19. 12. 11. 12. 40. 17. 39. 13. 43. 31.
 33. 18. 40. 43. 31. 47. 28. 25. 20. 33. 18. 24.]
pos data: (30, 592) char data: (30, 23884) word data: (30, 38443)
lexical diversity: (5, 1)
train shape: (30, 62920) test shape: (5, 62920)
training before feature selection: (30, 62920)
testing before feature selection: (5, 62920)
training after feature selection: (30, 53482)
testing after feature selection: (5, 53482)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (5,) (5,)
['candidate00004' 'candidate00004' 'candidate00004' 'candidate00004'
 'candidate00004']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.2
(30, 24084)
	 pos vocabulary size: 601 char vocabulary size: 24084
lexical diversity: (30, 1)
[29. 40. 36. 24. 41. 37. 12. 28. 19. 12. 11. 25. 40. 17. 39. 13. 43. 16.
 33. 18. 40. 43. 31. 38. 28. 25. 20. 33. 18. 39.]
pos data: (30, 601) char data: (30, 24084) word data: (30, 38670)
lexical diversity: (5, 1)
train shape: (30, 63356) test shape: (5, 63356)
training before feature selection: (30, 63356)
testing before feature selection: (5, 63356)
training after feature selection: (30, 53852)
testing after feature selection: (5, 53852)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


y_pred y_test: (5,) (5,)
['candidate00001' 'candidate00003' 'candidate00003' 'candidate00001'
 'candidate00005']
['candidate00001' 'candidate00002' 'candidate00003' 'candidate00004'
 'candidate00005']
accuracy: 0.6
problem00002 MEAN ACCURACY SCORES: 0.43999999999999995
MEAN SCORES ACCROSS PROBLEMS: 0.24499999999999997
elapsed time: 484.9901819229126


In [ ]:
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': False, 'c':0.1, 'feat_sel_percent': None, 'clf': 'SVC'}
params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':0.1, 'feat_sel_percent': 85, 'clf': 'SVC', 'calibration': True}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':1, 'feat_sel_percent': 85, 'clf': 'SVC'}
#params = {'n': 5,'ft': 3,'pt': 0.05,'feature_selection': True, 'c':10, 'feat_sel_percent': 85, 'clf': 'SVC'}
baseline(base_dir,out_dir,**params)
%timeit

In [110]:
help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model.logistic:

class LogisticRegression(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.linear_model.base.SparseCoefMixin)
 |  LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='warn', max_iter=100, multi_class='warn', verbose=0, warm_start=False, n_jobs=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the cross-
 |  entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 'sag' and 'lbfgs' solvers. It c

In [131]:
def baseline_logreg(path, outpath, param_grids):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    for index,problem in enumerate(problems):
        params = param_grids[index]
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        train_docs=[]
        for candidate in candidates:
            train_docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = [label for i,(text,label) in enumerate(train_docs)]
        #### Feature Extraction ###
        ###### Fit-Transform Training Set #######
        feat_extractor = Feature_Extractor(char=params['char'], pos=params['pos'], 
                                           word=params['word'], lexdiv=params['lexdiv'], 
                                           fnword=params['fnword'])
        train_data = feat_extractor.fit_transform(train_docs)
        ####### Feature Selection - Fit #######
        print("training before feature selection:", train_data.shape)
        # We use the default selection function: the 10% most significant features
        sel = SelectPercentile(f_classif, percentile=params['percentile'])
        train_data = sel.fit_transform(train_data, train_labels)
        print("training after feature selection:", train_data.shape)
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(train_docs), 'known texts')
        
        ###### Transform Test Set #######
        test_docs=read_files(path+os.sep+problem,unk_folder)
        test_data = feat_extractor.transform(test_docs)
        ####### Feature Selection #######
        print("test before feature selection:", test_data.shape)
        test_data = sel.transform(test_data)
        print("test after feature selection:", test_data.shape)
        print('\t', len(test_docs), 'unknown texts')
        
        ###### Applying Classifiers #####
        max_abs_scaler = preprocessing.MaxAbsScaler()
        scaled_train_data = max_abs_scaler.fit_transform(train_data)
        scaled_test_data = max_abs_scaler.transform(test_data)
        clf=CalibratedClassifierCV(LogisticRegression(multi_class=params['multi_class'],
                               solver=params['solver'], random_state=442
                              ))
        clf.fit(scaled_train_data, train_labels)
        predictions=clf.predict(scaled_test_data)
        # Saving output data
        out_data=[]
        unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
        pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
        for i,v in enumerate(predictions):
            out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
        with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
            json.dump(out_data, f, indent=4)
        print('\t', 'answers saved to file','answers-'+problem+'.json')
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'

param_grids = [{
    'char': True, # .855 with this combination
    'pos': False,
    'word': True,
    'lexdiv': True,
    'fnword': True,
    'percentile': 70, # 0.855 score
    'multi_class':'multinomial',
    'solver': 'lbfgs',
    },
    {
    'char': True,
    'pos': False,
    'word': True,
    'lexdiv': True,
    'fnword': True,
    'percentile': 10,  # 0.98 score
    'multi_class':'multinomial',
    'solver': 'lbfgs',
    }
]

baseline_logreg(base_dir,out_dir,param_grids)
%timeit

problem00001
Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: True
size of features: (140, 38084)
training before feature selection: (140, 38084)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


training after feature selection: (140, 26659)
	 language:  en
	 20 candidate authors
	 140 known texts
Extracting features from test data
lexdiv: True char: True pos: False word: True fnword: True
size of features: (105, 38084)
test before feature selection: (105, 38084)
test after feature selection: (105, 26659)
	 105 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


	 answers saved to file answers-problem00001.json
problem00002
Extracting features from train data
char vocabulary size: 14415
lexdiv: True char: True pos: False word: True fnword: True
size of features: (35, 18878)
training before feature selection: (35, 18878)
training after feature selection: (35, 1887)
	 language:  en
	 5 candidate authors
	 35 known texts
Extracting features from test data
lexdiv: True char: True pos: False word: True fnword: True
size of features: (21, 18878)
test before feature selection: (21, 18878)
test after feature selection: (21, 1887)
	 21 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


	 answers saved to file answers-problem00002.json
elapsed time: 10.462646245956421


In [123]:
def baseline_perceptron(path, outpath, param_grids):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    for index,problem in enumerate(problems):
        params = param_grids[index]
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        train_docs=[]
        for candidate in candidates:
            train_docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = [label for i,(text,label) in enumerate(train_docs)]
        #### Feature Extraction ###
        ###### Fit-Transform Training Set #######
        feat_extractor = Feature_Extractor(char=params['char'], pos=params['pos'], 
                                           word=params['word'], lexdiv=params['lexdiv'], 
                                           fnword=params['fnword'])
        train_data = feat_extractor.fit_transform(train_docs)
        ####### Feature Selection - Fit #######
        print("training before feature selection:", train_data.shape)
        # We use the default selection function: the 10% most significant features
        sel = SelectPercentile(f_classif, percentile=params['percentile'])
        train_data = sel.fit_transform(train_data, train_labels)
        print("training after feature selection:", train_data.shape)
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(train_docs), 'known texts')
        
        ###### Transform Test Set #######
        test_docs=read_files(path+os.sep+problem,unk_folder)
        test_data = feat_extractor.transform(test_docs)
        ####### Feature Selection #######
        print("test before feature selection:", test_data.shape)
        test_data = sel.transform(test_data)
        print("test after feature selection:", test_data.shape)
        print('\t', len(test_docs), 'unknown texts')
        
        ###### Applying Classifiers #####
        max_abs_scaler = preprocessing.MaxAbsScaler()
        scaled_train_data = max_abs_scaler.fit_transform(train_data)
        scaled_test_data = max_abs_scaler.transform(test_data)
        clf=CalibratedClassifierCV(MLPClassifier(random_state=442
                                                ))
        clf.fit(scaled_train_data, train_labels)
        predictions=clf.predict(scaled_test_data)
        # Saving output data
        out_data=[]
        unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
        pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
        for i,v in enumerate(predictions):
            out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
        with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
            json.dump(out_data, f, indent=4)
        print('\t', 'answers saved to file','answers-'+problem+'.json')
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'

param_grids = [{
    'char': True,
    'pos': False,
    'word': True,
    'lexdiv': True,
    'fnword': False,
    'percentile': 80, 
    },
    {
    'char': True,
    'pos': False,
    'word': True,
    'lexdiv': True,
    'fnword': False,
    'percentile': 70, 
    }
]

baseline_perceptron(base_dir,out_dir,param_grids)
%timeit

problem00001
Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: False
size of features: (140, 37905)
training before feature selection: (140, 37905)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


training after feature selection: (140, 30324)
	 language:  en
	 20 candidate authors
	 140 known texts
Extracting features from test data
lexdiv: True char: True pos: False word: True fnword: False
size of features: (105, 37905)
test before feature selection: (105, 37905)
test after feature selection: (105, 30324)
	 105 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


	 answers saved to file answers-problem00001.json
problem00002
Extracting features from train data
char vocabulary size: 14415
lexdiv: True char: True pos: False word: True fnword: False
size of features: (35, 18699)
training before feature selection: (35, 18699)
training after feature selection: (35, 13089)
	 language:  en
	 5 candidate authors
	 35 known texts
Extracting features from test data
lexdiv: True char: True pos: False word: True fnword: False
size of features: (21, 18699)
test before feature selection: (21, 18699)
test after feature selection: (21, 13089)
	 21 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


	 answers saved to file answers-problem00002.json
elapsed time: 11.59507966041565


In [125]:
def baseline_svm(path, outpath, param_grids):
    start_time = time.time()
    # Reading information about the collection
    infocollection = path+os.sep+'collection-info.json'
    problems = []
    language = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
            language.append(attrib['language'])
    for index,problem in enumerate(problems):
        params = param_grids[index]
        print(problem)
        # Reading information about the problem
        infoproblem = path+os.sep+problem+os.sep+'problem-info.json'
        candidates = []
        with open(infoproblem, 'r') as f:
            fj = json.load(f)
            unk_folder = fj['unknown-folder']
            for attrib in fj['candidate-authors']:
                candidates.append(attrib['author-name'])
        # Building training set
        train_docs=[]
        for candidate in candidates:
            train_docs.extend(read_files(path+os.sep+problem,candidate))
        train_labels = [label for i,(text,label) in enumerate(train_docs)]
        #### Feature Extraction ###
        ###### Fit-Transform Training Set #######
        feat_extractor = Feature_Extractor(char=params['char'], pos=params['pos'], 
                                           word=params['word'], lexdiv=params['lexdiv'], 
                                           fnword=params['fnword'])
        train_data = feat_extractor.fit_transform(train_docs)
        ####### Feature Selection - Fit #######
        print("training before feature selection:", train_data.shape)
        # We use the default selection function: the 10% most significant features
        sel = SelectPercentile(f_classif, percentile=params['percentile'])
        train_data = sel.fit_transform(train_data, train_labels)
        print("training after feature selection:", train_data.shape)
        print('\t', 'language: ', language[index])
        print('\t', len(candidates), 'candidate authors')
        print('\t', len(train_docs), 'known texts')
        
        ###### Transform Test Set #######
        test_docs=read_files(path+os.sep+problem,unk_folder)
        test_data = feat_extractor.transform(test_docs)
        ####### Feature Selection #######
        print("test before feature selection:", test_data.shape)
        test_data = sel.transform(test_data)
        print("test after feature selection:", test_data.shape)
        print('\t', len(test_docs), 'unknown texts')
        
        ###### Applying Classifiers #####
        max_abs_scaler = preprocessing.MaxAbsScaler()
        scaled_train_data = max_abs_scaler.fit_transform(train_data)
        scaled_test_data = max_abs_scaler.transform(test_data)
        clf=CalibratedClassifierCV(SVC(random_state=442))
        clf.fit(scaled_train_data, train_labels)
        predictions=clf.predict(scaled_test_data)
        # Saving output data
        out_data=[]
        unk_filelist = glob.glob(path+os.sep+problem+os.sep+unk_folder+os.sep+'*.txt')
        pathlen=len(path+os.sep+problem+os.sep+unk_folder+os.sep)
        for i,v in enumerate(predictions):
            out_data.append({'unknown-text': unk_filelist[i][pathlen:], 'predicted-author': v})
        with open(outpath+os.sep+'answers-'+problem+'.json', 'w') as f:
            json.dump(out_data, f, indent=4)
        print('\t', 'answers saved to file','answers-'+problem+'.json')
    print('elapsed time:', time.time() - start_time)

base_dir='pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02'
out_dir = base_dir+os.sep+'output-dir'
eval_dir = base_dir+os.sep+'eval-dir'

param_grids = [{
    'char': True,
    'pos': False,
    'word': True,
    'lexdiv': True,
    'fnword': True,
    'percentile': 60,
    },
    {
    'char': True,
    'pos': True,
    'word': False,
    'lexdiv': True,
    'fnword': True,
    'percentile': 40,
    }
]

baseline_svm(base_dir,out_dir,param_grids)
%timeit

problem00001
Extracting features from train data
char vocabulary size: 28470
lexdiv: True char: True pos: False word: True fnword: True
size of features: (140, 38084)
training before feature selection: (140, 38084)
training after feature selection: (140, 22850)
	 language:  en
	 20 candidate authors
	 140 known texts


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Extracting features from test data
lexdiv: True char: True pos: False word: True fnword: True
size of features: (105, 38084)
test before feature selection: (105, 38084)
test after feature selection: (105, 22850)
	 105 unknown texts


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change fro

	 answers saved to file answers-problem00001.json
problem00002
Extracting features from train data
char vocabulary size: 14415
	 pos vocabulary size: 619
lexdiv: True char: True pos: True word: False fnword: True
size of features: (35, 15214)
training before feature selection: (35, 15214)
training after feature selection: (35, 6085)
	 language:  en
	 5 candidate authors
	 35 known texts


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Extracting features from test data
lexdiv: True char: True pos: True word: False fnword: True
size of features: (21, 15214)
test before feature selection: (21, 15214)
test after feature selection: (21, 6085)
	 21 unknown texts
	 answers saved to file answers-problem00002.json
elapsed time: 9.585585594177246


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change fro

In [132]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
# Evaluation script for the Cross-Domain Authorship Attribution task @PAN2019.
We use the F1 metric (macro-average) as implemented in scikit-learn:
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
We include the following ad hoc rules:
- If authors are predicted which were not seen during training,
  these predictions will count as false predictions ('<UNK>' class)
  and they will negatively effect performance.
- If texts are left unattributed they will assigned to the ('<UNK>'
  class) and they will negatively effect performance.
- The <UNK> class is excluded from the macro-average across classes.
- If multiple test attributions are given for a single unknown document,
  only the first one will be taken into consideration.

Dependencies:
- Python 2.7 or 3.6 (we recommend the Anaconda Python distribution)
- scikit-learn

Usage from the command line:
>>> python pan19-cdaa-evaluator.py -i COLLECTION -a ANSWERS -o OUTPUT
where
    COLLECTION is the path to the main folder of the evaluation collection
    ANSWERS is the path to the answers folder of a submitted method
    OUTPUT is the path to the folder where the results of the evaluation will be saved

Example: 
>>>  python pan19-cdaa-evaluator.py -i ".\pan19-cross-domain-authorship-attribution-training-dataset-2019-01-23\" -a ".\answ
ers-unigram" -o ".\eval-unigram\"

# References:
@article{scikit-learn,
 title={Scikit-learn: Machine Learning in {P}ython},
 author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
         and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
         and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
         Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
 journal={Journal of Machine Learning Research},
 volume={12},
 pages={2825--2830},
 year={2011}
}
"""

import argparse
import os
import json
import warnings
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

def eval_measures(gt, pred):
    """Compute macro-averaged F1-scores, macro-averaged precision, 
    macro-averaged recall, and micro-averaged accuracy according the ad hoc
    rules discussed at the top of this file.
    Parameters
    ----------
    gt : dict
        Ground truth, where keys indicate text file names
        (e.g. `unknown00002.txt`), and values represent
        author labels (e.g. `candidate00003`)
    pred : dict
        Predicted attribution, where keys indicate text file names
        (e.g. `unknown00002.txt`), and values represent
        author labels (e.g. `candidate00003`)
    Returns
    -------
    f1 : float
        Macro-averaged F1-score
    precision : float
        Macro-averaged precision
    recall : float
        Macro-averaged recall
    accuracy : float
        Micro-averaged F1-score
    """

    actual_authors = list(gt.values())
    encoder = LabelEncoder().fit(['<UNK>'] + actual_authors)

    text_ids, gold_authors, silver_authors = [], [], []
    for text_id in sorted(gt):
        text_ids.append(text_id)
        gold_authors.append(gt[text_id])
        try:
            silver_authors.append(pred[text_id])
        except KeyError:
            # missing attributions get <UNK>:
            silver_authors.append('<UNK>')

    assert len(text_ids) == len(gold_authors)
    assert len(text_ids) == len(silver_authors)

    # replace non-existent silver authors with '<UNK>':
    silver_authors = [a if a in encoder.classes_ else '<UNK>' 
                      for a in silver_authors]

    gold_author_ints = encoder.transform(gold_authors)
    silver_author_ints = encoder.transform(silver_authors)

    # get F1 for individual classes (and suppress warnings):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        labels=list(set(gold_author_ints))
        # Exclude the <UNK> class
        for x in labels:
            if encoder.inverse_transform(np.array([x]))=='<UNK>':
                labels.remove(x)
        f1 = f1_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        precision = precision_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        recall = recall_score(gold_author_ints,
                  silver_author_ints,
                  labels,
                  average='macro')
        accuracy = accuracy_score(gold_author_ints,
                  silver_author_ints)

    return f1,precision,recall

def evaluate(ground_truth_file,predictions_file):
    # Calculates evaluation measures for a single attribution problem
    gt = {}
    with open(ground_truth_file, 'r') as f:
        for attrib in json.load(f)['ground_truth']:
            gt[attrib['unknown-text']] = attrib['true-author']

    pred = {}
    with open(predictions_file, 'r') as f:
        for attrib in json.load(f):
            if attrib['unknown-text'] not in pred:
                pred[attrib['unknown-text']] = attrib['predicted-author']
    f1,precision,recall =  eval_measures(gt,pred)
    return round(f1,3), round(precision,3), round(recall,3)

def evaluate_all(path_collection,path_answers,path_out):
    # Calculates evaluation measures for a PAN-18 collection of attribution problems
    infocollection = path_collection+os.sep+'collection-info.json'
    problems = []
    data = []
    with open(infocollection, 'r') as f:
        for attrib in json.load(f):
            problems.append(attrib['problem-name'])
    scores=[];
    for problem in problems:
        #prob_data = deepcopy(params)
        f1,precision,recall=evaluate(path_collection+os.sep+problem+os.sep+'ground-truth.json',path_answers+os.sep+'answers-'+problem+'.json')
        scores.append(f1)
        #prob_data.update({'problem-name': problem, 'macro-f1': round(f1,3), 'macro-precision': round(precision,3), 'macro-recall': round(recall,3)})
        #if os.path.isfile('metrics.csv'):
        #    with open('metrics.csv', 'a') as f:  # Just use 'w' mode in 3.x
        #        w = csv.DictWriter(f, prob_data.keys())
        #        w.writerow(prob_data)
        #else:
        #    with open('metrics.csv', 'w') as f:  # Just use 'w' mode in 3.x
        #        w = csv.DictWriter(f, prob_data.keys())
        #       w.writeheader()
        #        w.writerow(prob_data)
        #data.append(prob_data)
        print(str(problem),'Macro-F1:',round(f1,3))
    overall_score=sum(scores)/len(scores)
    # Saving data to output files (out.json and evaluation.prototext)
    with open(path_out+os.sep+'out.json', 'w') as f:
        json.dump({'problems': data, 'overall_score': round(overall_score,3)}, f, indent=4, sort_keys=True)
    print('Overall score:', round(overall_score,3))
    prototext='measure {\n key: "mean macro-f1"\n value: "'+str(round(overall_score,3))+'"\n}\n'
    with open(path_out+os.sep+'evaluation.prototext', 'w') as f:
        f.write(prototext)
    return # pd.read_csv('metrics.csv')
        
evaluate_all(base_dir,out_dir,eval_dir)
# logreg .568  0.623 with more features
# mlp 0.481
# svm 0.581

problem00001 Macro-F1: 0.498
problem00002 Macro-F1: 0.748
Overall score: 0.623


In [ ]:
df = pd.read_csv('metrics.csv')
df

In [ ]:
df.set_index('c', inplace=True)
df

In [ ]:
df[df['feature_selection'] == True].groupby(['problem-name'])['macro-recall', 'macro-precision'].plot(legend=True)